In [16]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import snowflake.connector
import os

# Thông tin kết nối Snowflake
conn_params = {
    'user': 'dohung24112002',
    'password': 'Doviethung24112002@',
    'account': 'uz36730.ap-southeast-1',
    'warehouse': 'COMPUTE_WH',
    'database': 'AIRBNB',
    'schema': 'DEV'
}

# Tạo kết nối tới Snowflake
conn = snowflake.connector.connect(**conn_params)
print('Connected Successfully!')

# Read recipe inputs
Target_Dataset = dataiku.Dataset("TARGET_DATASET")
Target_Dataset_df = Target_Dataset.get_dataframe()


# Bảng đích trong Snowflake
target_table = 'DWH_CHURN_DATASET'

# Đọc dữ liệu hiện có trong Snowflake
existing_data_query = f"SELECT * FROM {target_table}"
existing_data_df = pd.read_sql(existing_data_query, conn)
print(f"Existing data rows: {len(existing_data_df)}")

Connected Successfully!
Existing data rows: 3987


In [17]:
# Sử dụng applymap để viết hoa tất cả các giá trị
Target_Dataset_df.columns= Target_Dataset_df.columns.str.upper()
Target_Dataset_df

CUSTOMERID  GENDER SENIORCITIZEN PARTNER DEPENDENTS  TENURE PHONESERVICE MULTIPLELINES INTERNETSERVICE       ONLINESECURITY         ONLINEBACKUP     DEVICEPROTECTION          TECHSUPPORT          STREAMINGTV      STREAMINGMOVIES        CONTRACT PAPERLESSBILLING              PAYMENTMETHOD  MONTHLYCHARGES  TOTALCHARGES  NUMADMINTICKETS  NUMTECHTICKETS CHURN        STATES        COUNTRY
0     7469-LKBCI    Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No      Colorado  United States
1     8091-TTVAX    Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No       Montana  United States
2     0280-XJGEX    Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes         Maine  United States
3     3655-SNQYZ    Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No       Vermont  United States
4     8191-XWSZG    Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No      Delaware  United States
...          ...     ...           ...     ...        ...     ...          ...           ...             ...                  ...                  ...                  ...                  ...                  ...                  ...             ...              ...                        ...             ...           ...              ...             ...   ...           ...            ...
3982  0406-BPDVR  Female           Yes     Yes         No      54          Yes           Yes     Fiber optic                   No                  Yes                   No                   No                  Yes                  Yes        One year              Yes    Credit card (automatic)          101.50       5373.10                0               5   Yes   Connecticut  United States
3983  0618-XWMSS    Male            No      No        Yes      28          Yes            No     Fiber optic                  Yes                   No                   No                   No                   No                   No  Month-to-month              Yes  Bank transfer (automatic)           74.90       2068.55                0               0   Yes      Illinois  United States
3984  1395-WSWXR    Male            No      No         No      24          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No               Mailed check           20.75        487.05                0               0    No  Pennsylvania  United States
3985  6023-GSSXW  Female            No     Yes         No      69          Y

In [19]:
# Xác định các bản ghi cần cập nhật hoặc thêm mới
merged_df = pd.merge(Target_Dataset_df, existing_data_df, on='CUSTOMERID', how='outer', indicator=True)
to_update_df = merged_df[merged_df['_merge'] == 'both']
to_insert_df = merged_df[merged_df['_merge'] == 'left_only']



In [21]:
### UPDATE STEP ###
# Xác định các bản ghi cần cập nhật thực sự (các cột khác nhau)
cols_to_compare = [col.replace('_x', '') for col in to_update_df.columns if col.endswith('_x')]

for col in cols_to_compare:
    to_update_df[col + '_different'] = to_update_df[col + '_x'] != to_update_df[col + '_y']

# Lọc các bản ghi có ít nhất một cột khác nhau
to_update_df = to_update_df[to_update_df[[col + '_different' for col in cols_to_compare]].any(axis=1)]

# Loại bỏ các cột so sánh
to_update_df.drop(columns=[col + '_different' for col in cols_to_compare], inplace=True)

# ----------# 

print(f"Records to update: {len(to_update_df)}, Records to insert: {len(to_insert_df)}")

# Lọc các cột có hậu tố `_x`
to_update_df = to_update_df.filter(regex='_x$|customerID')
to_insert_df = to_insert_df.filter(regex='_x$|customerID')

# Đổi tên các cột để loại bỏ hậu tố `_x`
to_update_df.columns = [col.rstrip('_x') for col in to_update_df.columns]
to_insert_df.columns = [col.rstrip('_x') for col in to_insert_df.columns]

Records to update: 0, Records to insert: 0


In [9]:
# Xác định các bản ghi cần cập nhật thực sự (các cột khác nhau)
cols_to_compare = to_update_df[Target_Dataset_df.columns]
cols_to_compare

customerID  gender SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines InternetService       OnlineSecurity         OnlineBackup     DeviceProtection          TechSupport          StreamingTV      StreamingMovies        Contract PaperlessBilling              PaymentMethod  MonthlyCharges  TotalCharges  numAdminTickets  numTechTickets Churn        States        Country
0     7469-LKBCI    Male            No      No         No      16          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No    Credit card (automatic)           18.95        326.80                0               0    No      Colorado  United States
1     8091-TTVAX    Male            No     Yes         No      58          Yes           Yes     Fiber optic                   No                   No                  Yes                   No                  Yes                  Yes        One year               No    Credit card (automatic)          100.35       5681.10                0               0    No       Montana  United States
2     0280-XJGEX    Male            No      No         No      49          Yes           Yes     Fiber optic                   No                  Yes                  Yes                   No                  Yes                  Yes  Month-to-month              Yes  Bank transfer (automatic)          103.70       5036.30                5               4   Yes         Maine  United States
3     3655-SNQYZ    Male            No     Yes        Yes      69          Yes           Yes     Fiber optic                  Yes                  Yes                  Yes                  Yes                  Yes                  Yes        Two year               No    Credit card (automatic)          113.25       7895.15                0               0    No       Vermont  United States
4     8191-XWSZG    Male            No      No         No      52          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        One year               No               Mailed check           20.65       1022.95                0               0    No      Delaware  United States
...          ...     ...           ...     ...        ...     ...          ...           ...             ...                  ...                  ...                  ...                  ...                  ...                  ...             ...              ...                        ...             ...           ...              ...             ...   ...           ...            ...
3982  0406-BPDVR  Female           Yes     Yes         No      54          Yes           Yes     Fiber optic                   No                  Yes                   No                   No                  Yes                  Yes        One year              Yes    Credit card (automatic)          101.50       5373.10                0               5   Yes   Connecticut  United States
3983  0618-XWMSS    Male            No      No        Yes      28          Yes            No     Fiber optic                  Yes                   No                   No                   No                   No                   No  Month-to-month              Yes  Bank transfer (automatic)           74.90       2068.55                0               0   Yes      Illinois  United States
3984  1395-WSWXR    Male            No      No         No      24          Yes            No              No  No internet service  No internet service  No internet service  No internet service  No internet service  No internet service        Two year               No               Mailed check           20.75        487.05                0               0    No  Pennsylvania  United States
3985  6023-GSSXW  Female            No     Yes         No      69          Y

In [10]:
# Tạo cột 'Different' và đặt giá trị mặc định là 'No'
merged_df['Different'] = 'No'

# Liệt kê tất cả các cột mà bạn muốn so sánh
columns_to_compare = Target_Dataset_df.columns.tolist()

# So sánh từng cột và cập nhật cột 'Different' nếu có sự khác biệt
for col in columns_to_compare:
    # So sánh giá trị giữa hai cột tương ứng
    merged_df['Different'] = merged_df.apply(lambda row: 'Yes' if row[col] != row[f"{col}_existing"] else row['Different'], axis=1)

# In kết quả
print(merged_df[['customerID', 'Different'] + columns_to_compare + [f"{col}_existing" for col in columns_to_compare]].head())

KeyError: 'customerID_existing'

In [0]:

for col in cols_to_compare:
    to_update_df[col + '_different'] = to_update_df[col + '_x'] != to_update_df[col + '_y']

In [0]:


# Đường dẫn tệp CSV trên máy cục bộ
local_file_path = '/home/dataiku/dss/python_file/to_insert_df.csv'

# Lấy thư mục từ đường dẫn tệp
directory = os.path.dirname(local_file_path)

# Kiểm tra xem thư mục có tồn tại hay không, nếu không thì tạo mới
if not os.path.exists(directory):
    os.makedirs(directory)

# Lưu DataFrame đã chuẩn hóa thành tệp CSV
to_insert_df.to_csv(local_file_path, index=False)
print(to_insert_df.head())

### INCREMENTAL UPDATE STEPS ###
# Thực hiện Update các bản ghi
if not to_update_df.empty:
    update_query = """
    UPDATE {table}
    SET {set_clause}
    WHERE CUSTOMERID = %s
    """
    for _, row in to_update_df.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in row.index if col != 'customerID'])
        values = [row[col] for col in row.index if col != 'customerID'] + [row['customerID']]
        conn.cursor().execute(update_query.format(table=target_table, set_clause=set_clause), values)


# Thực hiện Insert các bản ghi mới
# Tên stage và tệp trên Snowflake
stage_name = 'DWH_STAGE'
stage_file_path = f'@{stage_name}/to_insert_df.csv'

# Tạo stage nếu chưa tồn tại
conn.cursor().execute(f"CREATE OR REPLACE STAGE {stage_name}")

# Tải tệp CSV lên Snowflake Stage
conn.cursor().execute(f"PUT file://{local_file_path} {stage_file_path}")

# Chèn dữ liệu từ stage vào bảng
copy_query = f"""
COPY INTO {target_table}
FROM {stage_file_path}
FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1)
"""
conn.cursor().execute(copy_query)


# Đóng kết nối
conn.close()